# Desktop Vector Skin Version User Preferences

**Related Tasks:**

[Part 1 Task](https://phabricator.wikimedia.org/T260149)

[Part 2 Task](https://phabricator.wikimedia.org/T269664)


# Contents
1. [Calculate opt out rate among registered users](#Calculate-opt-out-rate-among-all-registered-users)
2. [Calculate opt out rate among active editors](#Calculate-opt-out-rate-among-active-editors)

## Data:

I reviewed the mediawiki [user_properties table](https://www.mediawiki.org/wiki/Manual:User_properties_table) to determine the current number of users that have a vector skin preference recorded for each of the early adopter wikis.

Some notes regarding this dataset:
* This reflects all current nondefault user preferences.  User property records are added to the database when they differ from their default value. The VectorSkinVersion record is created in the database when users who don't have that record change their preference but also when new user accounts are created, which makes it difficult to differentiate those who opted-in to latest Vector via a user preference change vs. someone whose account was created and defaulted to latest Vector.
* Only accounts for logged-in users.
* It is not possible with current instrumentation to determine what default skin was presented to each user. 
* Data reflects the current state and does not account for users that have opt'd in and opt out multiple times since deployment.

In [84]:
import pandas as pd
import numpy as np

import datetime as dt

from wmfdata import hive, mariadb

# Calculate opt out rate among all registered users

We first calculated the percent of all registered users on the test wikis that have set their vector skin preference to 'legacy'. Please note that registered users likely includes a large proportion of users that are not active and have not visited the site while the desktop improvement features have been available.
 

In [351]:
query = """ 
-- find all non-vector users
With non_vector_users AS
(
SELECT
    up_user AS non_v_user
FROM user_properties
WHERE 
    up_property = 'skin'
    AND (up_value LIKE '%timeless%'
or up_value LIKE '%monobook%'
or up_value LIKE '%modern%'
or up_value LIKE '%minerva%')
)

SELECT 
  up_value AS skin, 
  SUM(CAST(non_vector_users.non_v_user IS NULL AS INT)) AS users
FROM user_properties
LEFT JOIN non_vector_users
-- remove users not on vector
    ON non_vector_users.non_v_user = user_properties.up_user
WHERE 
    up_property = 'VectorSkinVersion'
GROUP BY up_value 
"""

In [352]:
#define list of target wikis 
wikis = ['frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ]

In [353]:
up_skin=list()
for wiki in wikis:
    prefs = mariadb.run(
      query.format(),
      wiki
    )
    up_skin.append(prefs)

skin= pd.concat(up_skin)

In [354]:
# replace skin field as string type for easier renaming 
skin['skin'] = skin['skin'].astype(str)


In [355]:
skin_aliases = {
    "b\'0\'":"unknown",
    "b\'1\'":"legacy",
    "b\'2\'":"modern"
}

skin= skin.replace({"skin": skin_aliases})

In [356]:
skin_users = skin['users'].sum()
print('Total number of users for whom we have vector skin preferences set in the user_properties table:' , skin_users)

Total number of users for whom we have vector skin preferences set in the user_properties table: 1255625.0


## Number of users for each skin type overall

In [357]:
user_skin=skin.groupby('skin').sum()
user_skin

,users
skin,
legacy,482270.0
modern,773296.0
unknown,59.0


There are currently 59 users (0.005%) with VectorSkinPreference set to a value of 0 instead of 1 (legacy) or 2(modern). I believe these cases are users that reset their global preferences or switched to a non-vector skin. 

However, this is a very small percentage of cases and for the purpose of this analysis, we are only focusing on users that explicitly changed their preference back to legacy.

## Number of users for each skin type by wiki

In [358]:
#Manual list of wikis to correspond to data values 
# FIXME: Need to adjust code to automatically link wiki name to value
wikis_list = ['frwiktionary', 'frwiktionary', 'frwiktionary', 'hewiki', 'hewiki', 'hewiki', 
              'ptwikiversity', 'ptwikiversity', 'frwiki', 'frwiki', 'frwiki', 'euwiki', 'euwiki',
             'euwiki', 'fawiki', 'fawiki', 'fawiki', 'ptwiki', 'ptwiki', 'ptwiki', 'kowiki', 'kowiki',
             'kowiki', 'trwiki','trwiki', 'trwiki', 'srwiki', 'srwiki', 'srwiki', 
              'bnwiki','bnwiki','bnwiki', 'dewikivoyage', 'dewikivoyage', 'dewikivoyage', 'vecwiki',
             'vecwiki','vecwiki']

In [359]:
skin['wiki'] = wikis_list

In [360]:
user_skin_bywiki=pd.pivot_table(skin, index=['wiki','skin'],values=['users'],aggfunc=np.sum)

In [361]:
user_skin_bywiki

users
wiki          skin             
bnwiki        legacy    40411.0
              modern    16409.0
              unknown       3.0
dewikivoyage  legacy     2731.0
              modern     1194.0
              unknown       2.0
euwiki        legacy     2110.0
              modern    11590.0
              unknown       1.0
fawiki        legacy    21600.0
              modern   102916.0
              unknown       4.0
frwiki        legacy    72358.0
              modern   288473.0
              unknown      20.0
frwiktionary  legacy     4760.0
              modern    21387.0
              unknown       6.0
hewiki        legacy    28240.0
              modern   190672.0
              unknown       4.0
kowiki        legacy    46331.0
              modern    20320.0
              unknown       6.0
ptwiki        legacy   145184.0
              modern    70933.0
              unknown       9.0
ptwikiversity legacy      484.0
              modern     3002.0
srwiki        legacy    15998.0
              modern     7762.0
              unknown       2.0
trwiki        legacy   100009.0
              modern    37617.0
              unknown       2.0
vecwiki       legacy     2054.0
              modern     1021.0
              unknown       0.0

## Total Number of Registered Users on Test Wikis

We can use the total number of registered users on test wikis using the [mediawiki user table](https://www.mediawiki.org/wiki/Manual:User_table) to estimate the opt-out rate.

In [362]:
# collect total number of users on each wiki

query = """ 
SELECT 
    COUNT(DISTINCT user_id) AS num_users
FROM user"""


In [363]:
user_count = mariadb.run(commands = query, dbs = wikis, format="pandas")

In [364]:
user_count['wiki'] = wikis

In [365]:
user_count

,num_users,wiki
0,309333,frwiktionary
1,854858,hewiki
2,32000,ptwikiversity
3,4143491,frwiki
4,125504,euwiki
5,1043757,fawiki
6,2618458,ptwiki
7,690638,kowiki
8,1315710,trwiki
9,288550,srwiki


## Opt Out Rate for Registered Users

The opt-out rate was calculated by dividing the total number of users with their vector version preference set to 'legacy' by the total number of all registered users on the wiki. 

In [366]:
# Create list of legacy users - these are all users that opt-out assuming modern is the default

legacy_users = skin[skin['skin']=="legacy"]
#rename colums

legacy_users.columns = ['skin', 'num_legacy_users', 'wiki']


In [367]:
# join to user_count table to obtain opt-out rate for each wiki

opt_out_rate = legacy_users.merge(user_count, left_on = 'wiki', right_on = 'wiki')


In [368]:
# Calculate opt-out rate

opt_out_rate['pct_opt_out_rate'] = opt_out_rate['num_legacy_users']/ opt_out_rate['num_users'] * 100

opt_out_rate

,skin,num_legacy_users,wiki,num_users,pct_opt_out_rate
0,legacy,4760.0,frwiktionary,309333,1.538795
1,legacy,28240.0,hewiki,854858,3.303473
2,legacy,484.0,ptwikiversity,32000,1.512500
3,legacy,72358.0,frwiki,4143491,1.746305
4,legacy,2110.0,euwiki,125504,1.681221
5,legacy,21600.0,fawiki,1043757,2.069447
6,legacy,145184.0,ptwiki,2618458,5.544637
7,legacy,46331.0,kowiki,690638,6.708435
8,legacy,100009.0,trwiki,1315710,7.601143
9,legacy,15998.0,srwiki,288550,5.544273


# Calculate opt out rate among active editors

I reviewed the opt-out rate among active editors (users that had 5 or more content edits overall in June). This was calculated by finding the percent of active editors for each wiki (obtained using data from [mediawiki history table](https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Edits/MediaWiki_history) that have `VectorSkinVersion` preference set to legacy in the [user properties table](https://www.mediawiki.org/wiki/Manual:User_properties_table/en). 

Since the modern vector version was deployed as default to all of the test wikis in June, it was assumed that any users with a non-default preference recorded as legacy have opt-d out. I filtered out any users that are not currently using vector.


In [271]:
HIVE_SNAPSHOT = "2021-06"
START_OF_DATA = "2021-06-01"
END_OF_DATA = "2021-07-01"

## Collect number of active editors

In [272]:
#all active editors from the past month

active_editor_query = """

WITH yr_proj_edits as (
    select
        event_user_text as `user`,
        event_user_id as user_id,
        wiki_db as proj,
        sum(if(wiki_db = "wikidatawiki", 0.1, 1)) as content_edits,
        max(event_timestamp) as latest_edit
    from wmf.mediawiki_history
    where
        -- review target wikis
        wiki_db IN ('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
                    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ) and
        -- REGISTERED
        event_user_is_anonymous = false and
        
        -- NON-BOT
        size(event_user_is_bot_by) = 0 and
        not array_contains(event_user_groups, "bot") and
        
        -- CONTENT EDITS
        event_entity = "revision" and
        event_type = "create" and
        page_namespace_is_content = true and
        
        -- FROM THE LAST YEAR
        event_timestamp >= "{START_OF_DATA}" and event_timestamp < "{END_OF_DATA}" and
        
        -- FROM THE LATEST SNAPSHOT
        snapshot = "{hive_snapshot}"
    
    -- PER USER, PER WIKI
    group by event_user_text, event_user_id, wiki_db
)

-- FINAL SELECT OF
select 
    `user` as user_name,
    user_id as user_id,
    proj as wiki,
    global_edits
from 
-- JOINED TO THEIR HOME WIKI AND GLOBAL EDITS
(
    select
        `user`,
        user_id,
        proj,
        -- in the unlikely event that wikis are tied by edit count and latest edit, 
        -- row_number() will break it somehow
        row_number() over (partition by `user` order by content_edits desc, latest_edit desc) as rank,
        sum(content_edits) over (partition by `user`) as global_edits
    from yr_proj_edits
) yr_edits
where
rank = 1
and global_edits =5
"""

In [273]:
active_editor = hive.run(
    active_editor_query.format(
        hive_snapshot = HIVE_SNAPSHOT,
        START_OF_DATA= START_OF_DATA,
        END_OF_DATA=END_OF_DATA
    )
)

In [274]:
#Total_active_ed = active_editor['user_id'].count()

Total_active_ed = active_editor.groupby(['wiki'])[['user_id']].count()

print('Total number of editors for whom we will be checking vector skin preferences:' , Total_active_ed)

Total number of editors for whom we will be checking vector skin preferences:                user_id
wiki                  
bnwiki              36
dewikivoyage         2
euwiki               9
fawiki             209
frwiki             614
frwiktionary        12
hewiki             122
kowiki              92
ptwiki             336
ptwikiversity        1
srwiki              20
trwiki             134


In [ ]:
Note: There are were no active editors on Venetian Wikipedia during the reviewed time period.

## Vector Skin Preferences By Active Users

In [336]:
#Querying user_properties for getting the skin preferences set by the active editors we got in the above query

query='''
-- find all non-vector users
With non_vector_users AS
(
SELECT
    up_user AS non_v_user
FROM user_properties
WHERE 
    up_property = 'skin'
    AND (up_value LIKE '%timeless%'
or up_value LIKE '%monobook%'
or up_value LIKE '%modern%'
or up_value LIKE '%minerva%')
)

SELECT 
  up_value AS skin, 
  SUM(CAST(non_vector_users.non_v_user IS NULL AS INT)) AS users
FROM user_properties
LEFT JOIN non_vector_users
-- remove users not on vector
    ON non_vector_users.non_v_user = user_properties.up_user
WHERE up_user in ({users})
    AND up_property = "VectorSkinVersion"
GROUP BY up_value
'''


In [337]:
#define list of target wikis
wikis = ['frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage' ]


In [338]:
# Looping through each wiki for the list of users for each skin

up_skin=list()
for wiki in wikis:
    user_ids = active_editor[active_editor['wiki'] == wiki]["user_id"]
    user_list = ','.join([str(u) for u in user_ids])
    prefs = mariadb.run(
      query.format(users=user_list),
      wiki
    )
    up_skin.append(prefs)

skin= pd.concat(up_skin)

In [340]:
#List of wikis to correspond to data values 
# FIXME: Need to adjust code to automatically link wiki name to value
wikis_list = ['frwiktionary', 'hewiki', 'hewiki', 'ptwikiversity', 'frwiki', 'frwiki', 'euwiki',
             'fawiki', 'fawiki', 'ptwiki', 'ptwiki', 'kowiki', 'kowiki', 'trwiki', 'trwiki', 'srwiki', 'srwiki', 'bnwiki', 'bnwiki',
             'dewikivoyage']

In [341]:
# add wiki column
skin['wiki'] = wikis_list

In [342]:
# skin_users = skin['users'].sum()

skin_users = skin.groupby(['wiki']).sum()

print('Total number of active editor for whom we have vector skin preferences set in the user_properties table:' , skin_users)

Total number of active editor for whom we have vector skin preferences set in the user_properties table:                users
wiki                
bnwiki          29.0
dewikivoyage     1.0
euwiki           5.0
fawiki         156.0
frwiki         349.0
frwiktionary     4.0
hewiki          75.0
kowiki          73.0
ptwiki         246.0
ptwikiversity    1.0
srwiki          12.0
trwiki         118.0


Note: There are a number of active editors who do not have data for vector skin preference in the user_preference table indicating that they are set to the default 'Modern' skin or they were deleted from the user_preference table. For the purpose of the analysis below, let's default them to 'Modern' skin.

In [343]:
modern_users=np.subtract(Total_active_ed,skin_users)
modern_users

,user_id
wiki,
bnwiki,7.0
dewikivoyage,1.0
euwiki,4.0
fawiki,53.0
frwiki,265.0
frwiktionary,8.0
hewiki,47.0
kowiki,19.0
ptwiki,90.0


In [344]:
modern_users['skin'] = 'modern'

In [345]:
# Reformat dataframe to match strucutre of skin data frame for joining
modern_users.reset_index(inplace=True)
modern_users = modern_users.rename(columns = {'index':'wiki'})

In [346]:
modern_users = modern_users.rename(columns={"user_id": "users"})

In [347]:
# covert skin column to string type 
skin['skin'] = skin['skin'].astype(str)

In [348]:
skin_aliases = {
    "b\'0\'":"unknown",
    "b\'1\'":"legacy",
    "b\'2\'":"modern"
}

skin= skin.replace({"skin": skin_aliases})

In [350]:
# combine modern_users with skin table
skin  = skin.append(modern_users,ignore_index=True)


## Number of Active Editors for Skin Type 

In [331]:
user_skin=skin.groupby('skin').sum()
user_skin

,users
skin,
legacy,184.0
modern,1403.0


In [332]:
user_skin_bywiki=pd.pivot_table(skin, index=['wiki','skin'],values=['users'],aggfunc=np.sum)
user_skin_bywiki

users
wiki          skin         
bnwiki        legacy   10.0
              modern   26.0
dewikivoyage  legacy    1.0
              modern    1.0
euwiki        modern    9.0
fawiki        legacy    8.0
              modern  201.0
frwiki        legacy   37.0
              modern  577.0
frwiktionary  modern   12.0
hewiki        legacy    8.0
              modern  114.0
kowiki        legacy   18.0
              modern   74.0
ptwiki        legacy   62.0
              modern  274.0
ptwikiversity modern    1.0
srwiki        legacy    7.0
              modern   13.0
trwiki        legacy   33.0
              modern  101.0

## Perentage of Active Editors for Each Skin Type

In [333]:
# overall
pct_user_skin=(100. * user_skin / user_skin.sum()).round(1).astype(str) + '%'
pct_user_skin.sort_values(by=['users'],ascending=False)

,users
skin,
modern,88.4%
legacy,11.6%


In [334]:
# by target wiki


pct_user_skin_bywiki = user_skin_bywiki.groupby(['wiki', 'skin']).agg({'users': 'sum'})
wiki = user_skin_bywiki.groupby(['wiki']).agg({'users': 'sum'})
pct_user_skin_bywiki.div(wiki, level='wiki') * 100


users
wiki          skin              
bnwiki        legacy   27.777778
              modern   72.222222
dewikivoyage  legacy   50.000000
              modern   50.000000
euwiki        modern  100.000000
fawiki        legacy    3.827751
              modern   96.172249
frwiki        legacy    6.026059
              modern   93.973941
frwiktionary  modern  100.000000
hewiki        legacy    6.557377
              modern   93.442623
kowiki        legacy   19.565217
              modern   80.434783
ptwiki        legacy   18.452381
              modern   81.547619
ptwikiversity modern  100.000000
srwiki        legacy   35.000000
              modern   65.000000
trwiki        legacy   24.626866
              modern   75.373134

The percentage of legacy users listed above for each wiki reflects the opt-out rate as the modern vector was presented as default for all these wikis.

The opt out rates among active editors for each target wiki are still below 40% except for German Wikivoyage which only had 2 active editors in June 2021- with 1 having a user preference set to legacy.

After German Wikivoyage, Serbian Wikipedia (srwiki) currently has the highest opt out rate (35%) among active editors on the test wikis. 